In [1]:
import numpy as np

In [2]:
# 对应状态集合Q
states = ('Healthy', 'Fever')
# 对应观测集合V
observations = ('normal', 'cold', 'dizzy')
# 初始状态概率向量π
start_probability = {'Healthy': 0.6, 'Fever': 0.4}
# 状态转移矩阵A
transition_probability = {
    'Healthy': {'Healthy': 0.7, 'Fever': 0.3},
    'Fever': {'Healthy': 0.4, 'Fever': 0.6},
}
# 观测概率矩阵B
emission_probability = {
    'Healthy': {'normal': 0.5, 'cold': 0.4, 'dizzy': 0.1},
    'Fever': {'normal': 0.1, 'cold': 0.3, 'dizzy': 0.6},
}

In [3]:
def generate_index_map(lables):
    id2label = {}
    label2id = {}
    i = 0
    for l in lables:
        id2label[i] = l
        label2id[l] = i
        i += 1
    return id2label, label2id
 
states_id2label, states_label2id = generate_index_map(states)
observations_id2label, observations_label2id = generate_index_map(observations)
print(states_id2label, states_label2id)
print(observations_id2label, observations_label2id)

{0: 'Healthy', 1: 'Fever'} {'Healthy': 0, 'Fever': 1}
{0: 'normal', 1: 'cold', 2: 'dizzy'} {'normal': 0, 'cold': 1, 'dizzy': 2}


In [4]:
def convert_map_to_vector(map_, label2id):
    """将概率向量从dict转换成一维array"""
    v = np.zeros(len(map_), dtype=float)
    for e in map_:
        v[label2id[e]] = map_[e]
    return v

 
def convert_map_to_matrix(map_, label2id1, label2id2):
    """将概率转移矩阵从dict转换成矩阵"""
    m = np.zeros((len(label2id1), len(label2id2)), dtype=float)
    for line in map_:
        for col in map_[line]:
            m[label2id1[line]][label2id2[col]] = map_[line][col]
    return m

In [5]:
A = convert_map_to_matrix(transition_probability, states_label2id, states_label2id)
print(A)
B = convert_map_to_matrix(emission_probability, states_label2id, observations_label2id)
print(B)
observations_index = [observations_label2id[o] for o in observations]
pi = convert_map_to_vector(start_probability, states_label2id)
print(pi)

[[0.7 0.3]
 [0.4 0.6]]
[[0.5 0.4 0.1]
 [0.1 0.3 0.6]]
[0.6 0.4]


In [6]:
# 随机生成观测序列和状态序列    
def simulate(T):

    def draw_from(probs):
        """
        1.np.random.multinomial:
        按照多项式分布，生成数据
        >>> np.random.multinomial(20, [1/6.]*6, size=2)
                array([[3, 4, 3, 3, 4, 3],
                       [2, 4, 3, 4, 0, 7]])
         For the first run, we threw 3 times 1, 4 times 2, etc.  
         For the second, we threw 2 times 1, 4 times 2, etc.
        2.np.where:
        >>> x = np.arange(9.).reshape(3, 3)
        >>> np.where( x > 5 )
        (array([2, 2, 2]), array([0, 1, 2]))
        """
        return np.where(np.random.multinomial(1,probs) == 1)[0][0]

    observations = np.zeros(T, dtype=int)
    states = np.zeros(T, dtype=int)
    states[0] = draw_from(pi)
    observations[0] = draw_from(B[states[0],:])
    for t in range(1, T):
        states[t] = draw_from(A[states[t-1],:])
        observations[t] = draw_from(B[states[t],:])
    return observations, states

In [7]:
# 生成模拟数据
observations_data, states_data = simulate(10)
print(observations_data)
print(states_data)
# 相应的label
print("病人的状态: ", [states_id2label[index] for index in states_data])
print("病人的观测: ", [observations_id2label[index] for index in observations_data])

[0 2 2 2 2 1 0 2 0 0]
[1 1 1 1 1 1 1 1 0 1]
病人的状态:  ['Fever', 'Fever', 'Fever', 'Fever', 'Fever', 'Fever', 'Fever', 'Fever', 'Healthy', 'Fever']
病人的观测:  ['normal', 'dizzy', 'dizzy', 'dizzy', 'dizzy', 'cold', 'normal', 'dizzy', 'normal', 'normal']


## offline 

In [8]:
def forward(obs_seq):
    """前向算法"""
    N = A.shape[0]
    T = len(obs_seq)
    
    # F保存前向概率矩阵
    F = np.zeros((N,T))
    F[:,0] = pi * B[:, obs_seq[0]]

    for t in range(1, T):
        for n in range(N):
            F[n,t] = np.dot(F[:,t-1], (A[:,n])) * B[n, obs_seq[t]]

    return F

def backward(obs_seq):
    """后向算法"""
    N = A.shape[0]
    T = len(obs_seq)
    # X保存后向概率矩阵
    X = np.zeros((N,T))
    X[:,-1:] = 1
#     print("x: ", X)

    for t in reversed(range(T-1)):
        for n in range(N):
#             dd = X[:,t+1] * A[n,:] * B[:, obs_seq[t+1]]
#             print(dd)
            X[n,t] = np.sum(X[:,t+1] * A[n,:] * B[:, obs_seq[t+1]])

    return X

In [9]:
forward(observations_data)

array([[3.00000000e-01, 2.26000000e-02, 4.31800000e-03, 1.44994000e-03,
        5.45750200e-04, 8.34294664e-04, 4.55160673e-04, 3.48202912e-05,
        3.38991655e-05, 1.33763535e-05],
       [4.00000000e-02, 6.84000000e-02, 2.86920000e-02, 1.11063600e-02,
        4.25927880e-03, 8.15787702e-04, 7.39761020e-05, 1.08560318e-04,
        7.55822781e-06, 1.47046863e-06]])

In [10]:
backward(observations_data)

array([[4.01764297e-05, 1.05055295e-04, 2.77630401e-04, 7.95876512e-04,
        3.58869440e-03, 1.03786400e-02, 2.63440000e-02, 1.40800000e-01,
        3.80000000e-01, 1.00000000e+00],
       [6.98473299e-05, 1.82347550e-04, 4.75673151e-04, 1.23288358e-03,
        3.02593280e-03, 7.58528000e-03, 3.86080000e-02, 9.16000000e-02,
        2.60000000e-01, 1.00000000e+00]])

In [11]:
def baum_welch_train(observations, A, B, pi, criterion=0.05):
    """无监督学习算法——Baum-Weich算法"""
    n_states = A.shape[0]
    n_samples = len(observations)

    done = False
    while not done:
        # alpha_t(i) = P(O_1 O_2 ... O_t, q_t = S_i | hmm)
        # Initialize alpha
        alpha = forward(observations)

        # beta_t(i) = P(O_t+1 O_t+2 ... O_T | q_t = S_i , hmm)
        # Initialize beta
        beta = backward(observations)
        # ξ_t(i,j)=P(i_t=q_i,i_{i+1}=q_j|O,λ)
        xi = np.zeros((n_states,n_states,n_samples-1))
        for t in range(n_samples-1):
            denom = np.dot(np.dot(alpha[:,t].T, A) * B[:,observations[t+1]].T, beta[:,t+1])
            for i in range(n_states):
                numer = alpha[i,t] * A[i,:] * B[:,observations[t+1]].T * beta[:,t+1].T
                xi[i,:,t] = numer / denom

        # γ_t(i)：gamma_t(i) = P(q_t = S_i | O, hmm)
        gamma = np.sum(xi,axis=1)
        # Need final gamma element for new B
        # xi的第三维长度n_samples-1，少一个，所以gamma要计算最后一个
        prod =  (alpha[:,n_samples-1] * beta[:,n_samples-1]).reshape((-1,1))
        gamma = np.hstack((gamma,  prod / np.sum(prod))) #append one more to gamma!!!
        
        # 更新模型参数
        newpi = gamma[:,0]
        newA = np.sum(xi,2) / np.sum(gamma[:,:-1],axis=1).reshape((-1,1))
        newB = np.copy(B)
        num_levels = B.shape[1]
        sumgamma = np.sum(gamma,axis=1)
        for lev in range(num_levels):
            mask = observations == lev
            newB[:,lev] = np.sum(gamma[:,mask],axis=1) / sumgamma
        
        # 检查是否满足阈值
        if np.max(abs(pi - newpi)) < criterion and \
                        np.max(abs(A - newA)) < criterion and \
                        np.max(abs(B - newB)) < criterion:
            done = 1
        A[:], B[:], pi[:] = newA, newB, newpi
    return newA, newB, newpi

In [12]:
A = np.array([[0.5, 0.5],[0.5, 0.5]])
B = np.array([[0.3, 0.3, 0.4],[0.3, 0.3, 0.4]])
pi = np.array([0.4, 0.6])
print("pi: ", pi)
pic = pi.copy()

observations_data, states_data = simulate(200)
newA, newB, newpi = baum_welch_train(observations_data, A, B, pic,0.02)
print("newA: ", newA)
print("newB: ", newB)
print("newpi: ", newpi)
print("pi: ", pi)


pi:  [0.4 0.6]
newA:  [[0.50000479 0.49999521]
 [0.50000588 0.49999412]]
newB:  [[0.31863893 0.26553042 0.41583065]
 [0.32135835 0.26447064 0.41417101]]
newpi:  [0.39898043 0.60101957]
pi:  [0.4 0.6]


## online
Online Learning with Hidden Markov Models[ Gianluigi Mongillo ,Sophie Deneve]

Start with: 
- initial guess for the model parameters, θ(0) = [A,B], 
- initial state probabilities, ˆql (0) ≡ P(x0 = l), 
- sufficient statistics, ˆφh ijk(0)

specific data struct


In [109]:
# 初始状态概率向量π
pi = np.array([0.5, 0.5])
print("初始状态概率向量π: ", pi)

初始状态概率向量π:  [0.5 0.5]


In [110]:
# 初始状态转移矩阵和观测矩阵
A = np.array([[0.5, 0.5],[0.5, 0.5]])
B = np.array([[0.333, 0.333, 0.333],[0.333, 0.333, 0.333]])
print("初始状态转移矩阵A:\n", A)
print("初始观测矩阵A:\n", B)


初始状态转移矩阵A:
 [[0.5 0.5]
 [0.5 0.5]]
初始观测矩阵A:
 [[0.333 0.333 0.333]
 [0.333 0.333 0.333]]


Fi:
  [[[[0.33 0.33 0.33]
   [0.33 0.33 0.33]]

  [[0.33 0.33 0.33]
   [0.33 0.33 0.33]]]


 [[[0.33 0.33 0.33]
   [0.33 0.33 0.33]]

  [[0.33 0.33 0.33]
   [0.33 0.33 0.33]]]]


In [115]:
# P(xt−1 = i, xt = j, xT−1 = l, xT = h, y0→T)
# which element to update in transition model or observation mode
i = 1 # xt-1
j = 0 # xt

# observed  
k = 0 #yT

# last two state
l = 0 # xT-1
h = 0 # xT

In [170]:
time_factor = 0.0001
print("time_factor: ", time_factor)

time_factor:  0.0001


In [116]:
def delta(i, j):
    if i == j:
        return 1
    else :
        return 0
def g_delta(i, j, l, h):
    return delta(i, l )*delta(j, h)

In [132]:
# γlh(yT) 
# probility P(xT-1 = l, xT = h | yT = k)
Gama = np.ones([2,2,3])

In [121]:
# 状态概率 ql (T − 1) ≡ P(xT−1 = l|y0→T−1)
# probility P(xT-1 = l | y0-T-1)
Q = pi
print("状态概率：\n", Q)

状态概率：
 [0.5 0.5]


In [124]:
# 信息概率
# P(xt-1 = i, xt = j, xT = h , yT = h)
Fi = 0.33*np.ones([2,2,2,3])
print("信息概率:\n ", Fi)

信息概率:
  [[[[0.33 0.33 0.33]
   [0.33 0.33 0.33]]

  [[0.33 0.33 0.33]
   [0.33 0.33 0.33]]]


 [[[0.33 0.33 0.33]
   [0.33 0.33 0.33]]

  [[0.33 0.33 0.33]
   [0.33 0.33 0.33]]]]


In [81]:

A[i][j] 
a1 = Fi[0,1,:,:].sum() /  Fi[0,:,:,:].sum()
print("a1: ", a1)

a1:  0.5


In [137]:
# initialise all probility with A B
Q0 = Q
print("Q0\n", Q0)

Q0
 [0.5 0.5]


In [210]:
class Q_data:
    def __init__(self,Q):
        self.Q = Q
    
    def updateQ(self, Gama):
        Q0 = self.Q
        self.Q[0] = (Gama[0][0][k] * Q0[0]) + (Gama[1][0][k] * Q0[1])  

        self.Q[1] = (Gama[0][1][k] * Q0[0]) + (Gama[1][1][k] * Q0[1])  

    def getQ(self):
        return self.Q

In [211]:
class Gama_data:
    def __init__(self, Gama):
        self.Gama = Gama
        self.prob_sum = None
        self.A = None
        self.B = None
    
    def updateModel(self, A, B):
        self.A = A
        self.B = B
        
    def updateGama(self, new_obs_k, Q_old):
        A = self.A
        B = self.B
        Q = Q_old
        prob_sum = (A[0][0]*B[0][k]*Q[0] + A[0][1]*B[1][k]*Q[0] + A[1][0]*B[0][k]*Q[1] + A[1][1]*B[1][k]*Q[1]  )
        self.prob_sum = prob_sum
        
        self.Gama[0][0][k] = (A[0][0]*B[0][k]) / prob_sum

        self.Gama[0][1][k] = (A[0][1]*B[1][k]) / prob_sum

        self.Gama[1][0][k] = (A[1][0]*B[0][k]) / prob_sum

        self.Gama[1][1][k] = (A[1][1]*B[1][k]) / prob_sum
        
    
    
    def get(self, i, j, k):
        return self.Gama[i][j][k]
        
    def getGama(self):
        return self.Gama
    
# how to define a datastruct
# member data
# method

# for Gama
# contatin latest Gama
# updata Gama from Q and yT, A, B
        

In [212]:
m_gama = Gama_data(Gama)

In [213]:
m_gama.updateModel(A,B)

In [214]:
m_gama.updateGama(new_obs_k=1,Q_old=Q)

In [215]:
m_gama.getGama()

array([[[1. , 0.5, 1. ],
        [1. , 0.5, 1. ]],

       [[1. , 0.5, 1. ],
        [1. , 0.5, 1. ]]])

In [216]:
m_q = Q_data(Q)

In [217]:
m_q.updateQ(Gama=m_gama.getGama())

In [218]:
m_q.getQ()

array([0.5, 0.5])

In [197]:
# update after get observation
# get new obs
k = 1

# update Gama(T) from Q(T-1)
# Gama(l, h, k)
prob_sum = (A[0][0]*B[0][k]*Q[0] + A[0][1]*B[1][k]*Q[0] + A[1][0]*B[0][k]*Q[1] + A[1][1]*B[1][k]*Q[1]  )
Gama[0][0][k] = (A[0][0]*B[0][k]) / prob_sum
Gama[0][1][k] = (A[0][1]*B[1][k]) / prob_sum
Gama[1][0][k] = (A[1][0]*B[0][k]) / prob_sum
Gama[1][1][k] = (A[1][1]*B[1][k]) / prob_sum


# update internal state posibility
# Q(T) = Gama(T)*Q(T-1)

Q[0]
Q[0] = (Gama[0][0][k] * Q0[0]) + (Gama[1][0][k] * Q0[1])  
Q[1] = (Gama[0][1][k] * Q0[0]) + (Gama[1][1][k] * Q0[1])  
print("q0: ", q0)
print("q1: ", q1)

q0:  0.5
q1:  0.5


In [198]:
# P(xt-1 = i, xt = j, xT = h , yT = k)
# updata enven corespond yT is not observed
# Fi[0][0][0][0] = 0.0
# Fi[i][j][h][k] = Gama[l][h][k] * Fi[i][j][l][k]
Fi[0][0][0][0]  = (Gama[0][0][k] * (Fi[0][0][0][0] + time_factor*(delta(0, k)*g_delta(0, 0 ,0, 0)*Q[0] - Fi[0][0][0][0] ) ) ) + \
     (Gama[1][0][k] * (Fi[0][0][1][0] + time_factor*(delta(0, k)*g_delta(0, 0 ,1, 0)*Q[0] - Fi[0][0][1][0] ) ) )

Fi[0][0][0][0] 

0.3299239529051302

In [177]:
Fi[0][0][0][0] 

0.33

In [103]:



b1 = Fi[:,1,:,1].sum() /  Fi[:,1,:,:].sum()
print("b1: ", b1)

In [86]:
l = 0
h = 1

In [104]:
Gama = np.ones([2,2])

In [105]:
Gama[0][0] = A[0][1] * B[0][1] / (A[0][1] * B[1][1]*Q[0] + A[0][0] * B[0][1]*Q[0] + A[1][0] * B[0][1]*Q[1] + A[1][1] * B[1][1]*Q[1] )

In [106]:
Gama

array([[0.5, 1. ],
       [1. , 1. ]])

In [97]:
f1 = Fi[0][0][0][0]

In [94]:
f1 = (Gama[0] + Gama[1]) * (f1 + time_factor* )

0.5

In [ ]:
Fi[0][0][0][0] = (Gama[0] *  ) + ()

In [166]:
class TT:
    def __init__(self):
        self.k = 44.4
        pass
    def p(self):
        print(self.k)

In [167]:
t = TT()

In [168]:
t.p()

44.4
